# Hydropower Simulations with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways.
This example demonstrates a few of the options for modeling hydropower generation.

## Dependencies

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))

"/Users/cbarrows/Documents/repos/SIIPExamples.jl"

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using Cbc # solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.05)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.05])

### Data
PowerSystems.jl links to some meaningless test data that is suitable for this example.
The [make_hydro_data.jl](../../script/3_PowerSimulations_examples/make_hydro_data.jl) script
loads three systems suitable for the examples here.

In [5]:
include(joinpath(pkgpath, "script", "3_PowerSimulations_examples", "make_hydro_data.jl"))

┌ Info: Parsing csv data in Hydro_Upstream_Input.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed Hydro_Upstream_Input.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cb

This line just overloads JuMP printing to remove double underscores added by PowerSimulations.jl

In [6]:
PSI.JuMP._wrap_in_math_mode(str) = "\$\$ $(replace(str, "__"=>"")) \$\$"

## Two PowerSimulations features determine hydropower representation.
There are two principal ways that we can customize hydropower representation in
PowerSimulations. First, we can play with the formulation applied to hydropower generators
using the `DeviceModel`. We can also adjust how simulations are configured to represent
different decision making processes and the information flow between those processes.

### Hydropower `DeviceModel`s

First, the assignment of device formulations to particular device types gives us control
over the representation of devices. This is accomplished by defining `DeviceModel`
instances. For hydro power representations, we have two available generator types in
PowerSystems:

In [7]:
TypeTree(PowerSystems.HydroGen)

HydroGen
├──HydroDispatch
├──HydroEnergyReservoir
└──HydroPumpedStorage

And in PowerSimulations, we have several available formulations that can be applied to
the hydropower generation devices:

In [8]:
TypeTree(PSI.AbstractHydroFormulation, scopesep = "\n", init_expand = 5)

PowerSimulations
AbstractHydroFormulation
├──PowerSimulations
│  AbstractHydroDispatchFormulation
│  ├──PowerSimulations
│  │  HydroDispatchRunOfRiver
│  └──PowerSimulations
│     AbstractHydroReservoirFormulation
│     ├──PowerSimulations
│     │  HydroDispatchReservoirStorage (0 children)
│     ├──PowerSimulations
│     │  HydroDispatchPumpedStoragewReservation (0 children)
│     ├──PowerSimulations
│     │  HydroDispatchReservoirBudget (0 children)
│     └──PowerSimulations
│        HydroDispatchPumpedStorage (0 children)
└──PowerSimulations
   AbstractHydroUnitCommitment
   ├──PowerSimulations
   │  HydroCommitmentReservoirStorage
   ├──PowerSimulations
   │  HydroCommitmentReservoirBudget
   └──PowerSimulations
      HydroCommitmentRunOfRiver

Let's see what some of the different combinations create. First, let's apply the
`HydroDispatchRunOfRiver` formulation to the `HydroEnergyReservoir` generators, and the
`FixedOutput` formulation to `HydroDispatch` generators.
 - The `FixedOutput` formulation just acts
like a load subtractor, forcing the system to accept it's generation.
 - The `HydroDispatchRunOfRiver` formulation represents the the energy flowing out of
a reservoir. The model can choose to produce power with that energy or just let it spill by.

In [9]:
devices = Dict{Symbol, DeviceModel}(
    :Hyd1 => DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver),
    :Hyd2 => DeviceModel(HydroDispatch, FixedOutput),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy_uc, horizon = 24)

┌ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.SYSTEM_BASE = 0
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/base.jl:289



Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      Hyd1:
        device_type = HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Hyd2:
        device_type = HydroDispatch
        formulation = PowerSimulations.FixedOutput
  branches: 
  services: 


Now we can see the resulting JuMP model:

In [10]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 48
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 24 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 48 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

The first two constraints are the power balance constraints that require the generation
from the controllable `HydroEnergyReservoir` generators to be equal to the load (flat 10.0 for all time periods)
minus the generation from the `HydroDispatch` generators [1.97, 1.983, ...]. The 3rd and 4th
constraints limit the output of the `HydroEnergyReservoir` generator to the limit defined by the
`max_activepwoer` time series. And the last 4 constraints are the lower and upper bounds of
the `HydroEnergyReservoir` operating range.

Next, let's apply the `HydroDispatchReservoirBudget` formulation to the `HydroEnergyReservoir` generators.

In [11]:
devices = Dict{Symbol, DeviceModel}(
    :Hyd1 => DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirBudget),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy_uc, horizon = 24)

┌ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.SYSTEM_BASE = 0
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/base.jl:289



Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      Hyd1:
        device_type = HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirBudget
  branches: 
  services: 


And, the resulting JuMP model:

In [12]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 48
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 24 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Finally, let's apply the `HydroDispatchReservoirStorage` formulation to the `HydroEnergyReservoir` generators.

In [13]:
devices = Dict{Symbol, DeviceModel}(
    :Hyd1 => DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirStorage),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy_uc, horizon = 24)

┌ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.SYSTEM_BASE = 0
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/base.jl:289



Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      Hyd1:
        device_type = HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirStorage
  branches: 
  services: 


In [14]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 144
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 72 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 144 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 96 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

### Multi-Stage `SimulationSequence`
The purpose of a multi-stage simulation is to represent scheduling decisions consistently
with the time scales that govern different elements of power systems.

#### Multi-Day to Daily Simulation:
In the multi-day model, we'll use a really simple representation of all system devices
so that we can maintain computational tractability while getting an estimate of system
requirements/capabilities.

In [15]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroEnergyReservoir =>
        DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirStorage),
)
template_md = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

For the daily model, we can increase the modeling detail since we'll be solving shorter
problems.

In [16]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroEnergyReservoir =>
        DeviceModel(HydroEnergyReservoir, HydroDispatchReservoirStorage),
)
template_da = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

In [17]:
stages_definition = Dict(
    "MD" => Stage(
        GenericOpProblem,
        template_md,
        c_sys5_hy_wk,
        solver,
        system_to_file = false,
    ),
    "DA" => Stage(
        GenericOpProblem,
        template_da,
        c_sys5_hy_uc,
        solver,
        system_to_file = false,
    ),
)

Dict{String,PowerSimulations.Stage{PowerSimulations.GenericOpProblem}} with 2 entries:
  "DA" => Stage()…
  "MD" => Stage()…

This builds the sequence and passes the the energy dispatch schedule for the `HydroEnergyReservoir`
generator from the "MD" stage to the "DA" stage in the form of an energy limit over the
synchronized periods.

In [18]:
sequence = SimulationSequence(
    step_resolution = Hour(48),
    order = Dict(1 => "MD", 2 => "DA"),
    feedforward_chronologies = Dict(("MD" => "DA") => Synchronize(periods = 2)),
    horizons = Dict("MD" => 2, "DA" => 24),
    intervals = Dict("MD" => (Hour(48), Consecutive()), "DA" => (Hour(24), Consecutive())),
    feedforward = Dict(
        ("DA", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_source_stage = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
    ),
    cache = Dict(("MD", "DA") => StoredEnergy(HydroEnergyReservoir, PSI.ENERGY)),
    ini_cond_chronology = IntraStageChronology(),
);

In [19]:
file_path = tempdir()

sim = Simulation(
    name = "hydro",
    steps = 1,
    stages = stages_definition,
    stages_sequence = sequence,
    simulation_folder = file_path,
)

build!(sim)

PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

We can look at the "MD" Model

In [20]:
sim.stages["MD"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 44
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 16 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 12 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 44 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 40 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "DA" model

In [21]:
sim.stages["DA"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 528
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 192 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 122 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 122 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 528 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 480 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can execute the simulation by running the following command
```julia
sim_results = execute!(sim)
```

#### 3-Stage Simulation:

In [22]:
stages_definition = Dict(
    "MD" => Stage(
        GenericOpProblem,
        template_md,
        c_sys5_hy_wk,
        solver,
        system_to_file = false,
    ),
    "DA" => Stage(
        GenericOpProblem,
        template_da,
        c_sys5_hy_uc,
        solver,
        system_to_file = false,
    ),
    "ED" => Stage(
        GenericOpProblem,
        template_da,
        c_sys5_hy_ed,
        solver,
        system_to_file = false,
    ),
)

sequence = SimulationSequence(
    step_resolution = Hour(48),
    order = Dict(1 => "MD", 2 => "DA", 3 => "ED"),
    feedforward_chronologies = Dict(
        ("MD" => "DA") => Synchronize(periods = 2),
        ("DA" => "ED") => Synchronize(periods = 24),
    ),
    intervals = Dict(
        "MD" => (Hour(48), Consecutive()),
        "DA" => (Hour(24), Consecutive()),
        "ED" => (Hour(1), Consecutive()),
    ),
    horizons = Dict("MD" => 2, "DA" => 24, "ED" => 12),
    feedforward = Dict(
        ("DA", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_source_stage = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
        ("ED", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_source_stage = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
    ),
    cache = Dict(("MD", "DA") => StoredEnergy(HydroEnergyReservoir, PSI.ENERGY)),
    ini_cond_chronology = IntraStageChronology(),
);

In [23]:
sim = Simulation(
    name = "hydro",
    steps = 1,
    stages = stages_definition,
    stages_sequence = sequence,
    simulation_folder = file_path,
)

Simulation()


In [24]:
build!(sim)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Presolve 156 (-3144) rows, 708 (-4320) columns and 852 (-7464) elements
Perturbing problem by 0.001% of 1000000 - largest nonzero change 0.107699 ( 0.0080622579%) - largest zero change 0.10618892
0  Obj 841.75397 Primal inf 197.86429 (156)
78  Obj 17763.714 Primal inf 118.94677 (111)
156  Obj 31071.858 Primal inf 35.773738 (50)
205  Obj 34146.664
Optimal - objective value 34129.959
After Postsolve, objective 34129.959, infeasibilities - dual 53844.549 (275), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 34129.959
Optimal - objective value 34129.959
Optimal objective 34129.95865 - 205 iterations time 0.012, Presolve 0.00
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01



PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

We can look at the "MD" Model

In [25]:
sim.stages["MD"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 44
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 16 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 12 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 44 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 40 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "DA" model

In [26]:
sim.stages["DA"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 528
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 192 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 122 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 122 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 528 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 480 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "ED" model

In [27]:
sim.stages["ED"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 264
Objective function type: JuMP.GenericAffExpr{Float64,JuMP.VariableRef}
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 96 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 62 constraints
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 62 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 264 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 240 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can execute the simulation by running the following command
```julia
sim_results = execute!(sim)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*